<a href="https://colab.research.google.com/github/ToniKayo/iowa-liquor-forecasting/blob/main/Data_Mining_Co_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# For visualizations
%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
!pip install kagglehub

import os
import json


In [ ]:
!mkdir -p ~/.kaggle

api_token = {"username":"danielcomerford","key":"1f24ed06dae2283bad0cba7971e251b6"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

# Set appropriate permissions
!chmod 600 /root/.kaggle/kaggle.json

# Verify the authentication file exists
!ls -la ~/.kaggle

total 16
drwxr-xr-x 2 root root 4096 Apr  1 14:07 .
drwx------ 1 root root 4096 Apr  1 14:07 ..
-rw------- 1 root root   74 Apr  1 14:07 kaggle.json


In [ ]:
!kaggle datasets list


ref                                                               title                                               size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  --------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                 Chocolate Sales Data 📊🍫                            14473  2025-03-19 03:51:40.270000          15338        267  1.0              
adilshamim8/student-depression-dataset                            Student Depression Dataset                        467020  2025-03-13 03:12:30.423000           6890        108  1.0              
atharvasoundankar/global-housing-market-analysis-2015-2024        🏡 Global Housing Market Analysis (2015-2024)       18363  2025-03-18 05:21:22.723000           1282         22  1.0              
willianoliveiragibin

import pandas as pd

# Load the dataset from the correct path
df = pd.read_csv("/content/Iowa_Liquor_Sales.csv")

# Display basic info
df.info()
df.head()


In [ ]:
!kaggle datasets download -d residentmario/iowa-liquor-sales --unzip
!ls -la


Dataset URL: https://www.kaggle.com/datasets/residentmario/iowa-liquor-sales
License(s): CC0-1.0
total 3385352
drwxr-xr-x 1 root root       4096 Apr  1 14:08 .
drwxr-xr-x 1 root root       4096 Apr  1 14:04 ..
drwxr-xr-x 4 root root       4096 Mar 28 13:39 .config
-rw-r--r-- 1 root root 3466579785 Apr  1 14:08 Iowa_Liquor_Sales.csv
drwxr-xr-x 1 root root       4096 Mar 28 13:39 sample_data


In [ ]:
import glob
csv_files = glob.glob("Iowa_Liquor_Sales.csv")
print("Available CSV files:", csv_files)

# Determine the main sales data file
# Usually the largest file is the main dataset
file_sizes = {f: os.path.getsize(f) for f in csv_files}
for file, size in sorted(file_sizes.items(), key=lambda x: x[1], reverse=True):
    print(f"{file}: {size / (1024 * 1024):.2f} MB")

# Select the main sales file (typically the largest)
sales_file = sorted(file_sizes.items(), key=lambda x: x[1], reverse=True)[0][0]
sales_path = sales_file

# Since the file is large, use chunking to efficiently load it
# Start with a small sample to understand the structure
sample_sales_df = pd.read_csv(sales_path, nrows=10000)  # Load just 10,000 rows for initial inspection

# Examine the sample
print("Sample sales data shape:", sample_sales_df.shape)
sample_sales_df.head()

# Check column info for the sample
sample_sales_df.info()

# Check for missing values in the sample
sample_missing = sample_sales_df.isnull().sum()
print("Columns with missing values in sample:")
print(sample_missing[sample_missing > 0])

# Now determine which columns we actually need for our analysis
# This will help us reduce memory usage when loading the full dataset
# Correct the column names to match the CSV file
needed_columns = ['Date', 'Store Number', 'Store Name', 'Category', 'Category Name',
                  'Vendor Number', 'Vendor Name', 'Item Number', 'Item Description',
                  'Bottle Volume (ml)', 'Bottles Sold', 'Sale (Dollars)', 'Volume Sold (Liters)']

# Load the full dataset with only the needed columns
# We'll use chunking to handle the large file size
chunk_size = 500000  # Adjust based on your available RAM
chunks = []

for chunk in pd.read_csv(sales_path, usecols=needed_columns, chunksize=chunk_size):
    # Process each chunk as needed
    # For example, filter by date range if you don't need all the data
    chunks.append(chunk)

    # Print progress
    print(f"Loaded chunk, shape: {chunk.shape}")

    # Optional: limit to first few chunks during development
    # if len(chunks) >= 3:
    #     break

# Combine all chunks
sales_df = pd.concat(chunks)
print("Full sales data shape:", sales_df.shape)

# Check for basic information in the full dataset
sales_df.info()

# Check for missing values
sales_missing = sales_df.isnull().sum()
print("Columns with missing values:")
print(sales_missing[sales_missing > 0])

# Summary statistics
sales_df.describe()

Available CSV files: ['Iowa_Liquor_Sales.csv']
Iowa_Liquor_Sales.csv: 3305.99 MB
Sample sales data shape: (10000, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Invoice/Item Number    10000 non-null  object 
 1   Date                   10000 non-null  object 
 2   Store Number           10000 non-null  int64  
 3   Store Name             10000 non-null  object 
 4   Address                10000 non-null  object 
 5   City                   10000 non-null  object 
 6   Zip Code               10000 non-null  object 
 7   Store Location         10000 non-null  object 
 8   County Number          9987 non-null   float64
 9   County                 9987 non-null   object 
 10  Category               9918 non-null   float64
 11  Category Name          9917 non-null   object 
 12  Vendor Number          10000 non-null  in

,Store Number,Category,Vendor Number,Item Number,Bottle Volume (ml),Bottles Sold,Volume Sold (Liters)
count,1.259108e+07,1.258306e+07,1.259107e+07,1.259108e+07,1.259108e+07,1.259108e+07,1.259108e+07
mean,3.565216e+03,1.044710e+06,2.574911e+02,4.603682e+04,9.289402e+02,8.140392e+00,7.489119e+00
std,9.312721e+02,5.435094e+04,1.416175e+02,5.301684e+04,7.340448e+02,2.217891e+01,2.679149e+01
min,2.106000e+03,1.012200e+05,1.000000e+01,1.010000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.604000e+03,1.012210e+06,1.150000e+02,2.705600e+04,7.500000e+02,2.000000e+00,1.500000e+00
50%,3.704000e+03,1.031200e+06,2.600000e+02,3.817700e+04,7.500000e+02,4.000000e+00,3.000000e+00
75%,4.304000e+03,1.062310e+06,3.800000e+02,6.375500e+04,1.000000e+03,1.200000e+01,9.000000e+00
max,9.932000e+03,1.901200e+06,9.870000e+02,9.992750e+05,3.780000e+05,1.500000e+04,1.500000e+04


In [ ]:
sales_df.tail()


,Date,Store Number,Store Name,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Bottle Volume (ml),Bottles Sold,Sale (Dollars),Volume Sold (Liters)
12591072,10/31/2017,5423,Stammer Liquor Corp,1071100.0,Cocktails /RTD,305.0,Mhw LTD,73802,FrappaChata Iced Coffee Blend,800,3,$12.00,2.4
12591073,10/31/2017,5423,Stammer Liquor Corp,1011200.0,Straight Bourbon Whiskies,85.0,Brown Forman Corp.,20375,Old Forester Statesman,750,2,$41.22,1.5
12591074,10/31/2017,5423,Stammer Liquor Corp,1011500.0,Bottled in Bond Bourbon,85.0,Brown Forman Corp.,20372,Old Forester 1897 Whisky Row Series,750,2,$37.47,1.5
12591075,10/31/2017,5423,Stammer Liquor Corp,1011200.0,Straight Bourbon Whiskies,85.0,Brown Forman Corp.,20369,Old Forester 1870 Whisky Row Series,750,2,$33.74,1.5
12591076,10/31/2017,5423,Stammer Liquor Corp,1091100.0,American Distilled Spirit Specialty,481.0,Sugarlands Distilling Company LLC,77309,Sugarlands Shine Peanut Butter & Jelly Moonshine,750,2,$19.50,1.5


In [ ]:
sales_df.head()

,Date,Store Number,Store Name,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Bottle Volume (ml),Bottles Sold,Sale (Dollars),Volume Sold (Liters)
0,11/20/2015,2191,Keokuk Spirits,NaN,NaN,255.0,Wilson Daniels Ltd.,297,Templeton Rye w/Flask,750,6,$162.84,4.50
1,11/21/2015,2205,Ding's Honk And Holler,NaN,NaN,255.0,Wilson Daniels Ltd.,297,Templeton Rye w/Flask,750,12,$325.68,9.00
2,11/16/2015,3549,Quicker Liquor Store,NaN,NaN,130.0,Disaronno International LLC,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,150,2,$19.20,0.30
3,11/04/2015,2513,Hy-Vee Food Store #2 / Iowa City,NaN,NaN,65.0,Jim Beam Brands,237,Knob Creek w/ Crystal Decanter,1750,3,$160.02,5.25
4,11/17/2015,3942,Twin Town Liquor,NaN,NaN,130.0,Disaronno International LLC,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,150,2,$19.20,0.30


In [ ]:
# prompt: convert date to datetime

import pandas as pd
# Assuming 'sales_df' is your DataFrame and 'Date' is the column containing dates
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

# Example of accessing the datetime object for the first row
first_date = sales_df['Date'][0]
first_date


Timestamp('2015-11-20 00:00:00')

In [ ]:
sales_numeric_cols = sales_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
sales_df[sales_numeric_cols] = sales_df[sales_numeric_cols].fillna(sales_df[sales_numeric_cols].median())

#Categorical Data
sales_categorical_cols = ['Store Name', 'Category Name', 'Vendor Name', 'Item Description']

In [ ]:
sales_df.duplicated().sum()

np.int64(15689)

In [ ]:
sales_df.drop_duplicates(inplace=True)

In [ ]:
sales_df.isna().sum()

,0
Date,0
Store Number,0
Store Name,0
Category,0
Category Name,16054
Vendor Number,0
Vendor Name,1
Item Number,0
Item Description,0
Bottle Volume (ml),0


In [ ]:
# prompt: Show the item description column

print(sales_df['Item Description'])


0                                      Templeton Rye w/Flask
1                                      Templeton Rye w/Flask
2              Disaronno Amaretto Cavalli Mignon 3-50ml Pack
3                             Knob Creek w/ Crystal Decanter
4              Disaronno Amaretto Cavalli Mignon 3-50ml Pack
                                  ...                       
12591072                       FrappaChata Iced Coffee Blend
12591073                              Old Forester Statesman
12591074                 Old Forester 1897 Whisky Row Series
12591075                 Old Forester 1870 Whisky Row Series
12591076    Sugarlands Shine Peanut Butter & Jelly Moonshine
Name: Item Description, Length: 12575388, dtype: object


In [ ]:
sales_df['Item Description Lower'] = sales_df['Item Description'].str.lower()


In [ ]:
from collections import Counter
import re

# Join all descriptions into one string
all_text = ' '.join(sales_df['Item Description Lower'].dropna())

# Split into individual words (tokens), removing punctuation
words = re.findall(r'\b[a-z]+\b', all_text)

# Count the frequency of each word
word_counts = Counter(words)

# View the most common terms
print(word_counts.most_common(100))


[('vodka', 2746202), ('rum', 1196004), ('black', 854695), ('s', 834483), ('mini', 794883), ('whiskey', 643628), ('spiced', 638799), ('prf', 584898), ('canadian', 557568), ('liqueur', 541675), ('paramount', 526828), ('crown', 509962), ('velvet', 501440), ('pet', 474643), ('gin', 474049), ('captain', 473215), ('morgan', 473193), ('whisky', 448327), ('schnapps', 434952), ('tequila', 432650), ('smirnoff', 423898), ('five', 406059), ('hawkeye', 399310), ('brandy', 390594), ('royal', 368163), ('bacardi', 366458), ('jack', 357649), ('seagrams', 344802), ('uv', 335463), ('o', 324938), ('jose', 321837), ('cuervo', 321837), ('cinnamon', 289429), ('clock', 288260), ('dekuyper', 287927), ('beam', 286845), ('old', 285841), ('absolut', 266308), ('bl', 253684), ('vs', 251690), ('fireball', 243289), ('daniels', 239181), ('cherry', 235487), ('jim', 235262), ('reposado', 227646), ('lbl', 224373), ('white', 223288), ('mccormick', 216877), ('margarita', 215702), ('superior', 211533), ('silver', 209942), (

In [ ]:
core_types = [
    'vodka', 'rum', 'whiskey', 'whisky', 'bourbon', 'scotch', 'brandy',
    'gin', 'tequila', 'liqueur', 'schnapps', 'amaretto', 'moonshine',
    'mezcal', 'vermouth', 'absinthe', 'cognac', 'rye'
]

additional_types = [
    'wine', 'champagne', 'cider', 'port', 'sherry', 'sake'
]


flavor_keywords = [
    'cinnamon', 'peach', 'apple', 'grape', 'cherry', 'coffee', 'caramel',
    'peppermint', 'raspberry', 'vanilla', 'hazelnut', 'coconut', 'lime',
    'lemon', 'orange', 'spiced', 'sweet', 'sour'
]

package_keywords = [
    'mini', 'mignon', 'gift', 'flask', 'set', 'combo', 'sampler', 'traveler', 'pack', 'limited', 'reserve'
]

for word in core_types + additional_types + flavor_keywords + package_keywords:
    count = sales_df['Item Description'].str.contains(word, case=False, na=False).sum()
    if count > 0:
        print(f"'{word}': {count} rows")




'vodka': 2746202 rows
'rum': 1346437 rows
'whiskey': 643628 rows
'whisky': 448327 rows
'bourbon': 180433 rows
'scotch': 204401 rows
'brandy': 390594 rows
'gin': 671031 rows
'tequila': 432667 rows
'liqueur': 541675 rows
'schnapps': 434952 rows
'amaretto': 84623 rows
'moonshine': 10685 rows
'mezcal': 2567 rows
'absinthe': 1991 rows
'cognac': 90030 rows
'rye': 85008 rows
'wine': 354 rows
'cider': 4322 rows
'port': 44744 rows
'sherry': 378 rows
'sake': 11 rows
'cinnamon': 289429 rows
'peach': 194897 rows
'apple': 281679 rows
'grape': 65874 rows
'cherry': 237405 rows
'coffee': 86991 rows
'caramel': 81442 rows
'peppermint': 194958 rows
'raspberry': 192522 rows
'vanilla': 58351 rows
'hazelnut': 1321 rows
'coconut': 145824 rows
'lime': 112196 rows
'lemon': 66646 rows
'orange': 42189 rows
'spiced': 638799 rows
'sweet': 39383 rows
'sour': 64590 rows
'mini': 803436 rows
'mignon': 97 rows
'gift': 1756 rows
'flask': 38661 rows
'set': 2787 rows
'combo': 30 rows
'sampler': 740 rows
'traveler': 141775

In [ ]:
# Load all location weather
LA_weather_df = pd.read_csv("/content/weather_data_LA (1).csv")
NY_weather_df = pd.read_csv("/content/weather_data_NY (1).csv")
CH_weather_df = pd.read_csv("/content/weather_data_CH (1).csv")

location_weather = {"LA": LA_weather_df, "NY": NY_weather_df, "CH": CH_weather_df}

weather_data = pd.concat(location_weather, ignore_index=True)
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])
weather_data_july_dec = weather_data[weather_data['datetime'].dt.month > 6]


FileNotFoundError: [Errno 2] No such file or directory: '/content/weather_data_LA (1).csv'

In [ ]:
weather_data_july_dec_file = "/content/weather_data_July_December.csv"
weather_data_july_dec.to_csv(weather_data_jul_dec_file, index=False)

### **BASIC INFO**

In [ ]:
# Summary
sales_df.info()

In [ ]:
# Check missing values
print(sales_df.isnull().sum())


*   **Peripytpe** is missing 136796 values. After investigation, it was found that these values are missing because there was no rain or snow. Since precipitation only occurs under those conditions, no precipitation type was recorded. Replace with 'No Percipitation'.
*   **severerisk** is missing 149116 values. We can immdeiately delete this column as 100% of values are missing.




In [ ]:
df["preciptype"] = df["preciptype"].fillna("No Precipitation")
df.drop(columns=["severerisk"], inplace=True)

In [ ]:
#Recheck

# Check missing values
#print(df.isnull().sum())
#print(sales_df.describe())
sales_df.isnull().sum()


In [ ]:
# Drop columns that are irrelevant
df.drop(columns=['stations', 'transaction_datetime', 'datetime', 'icon', ], inplace=True)

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
sample_rows = df.sample(5, random_state=42)
print(sample_rows)

In [ ]:
#convert dates to the datetime format and extract year, month, day of week

df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d/%m/%Y')

df['year'] = df['transaction_date'].dt.year
df['month'] = df['transaction_date'].dt.month
df['day'] = df['transaction_date'].dt.dayofweek

In [ ]:
# Create a new column for sales revenue
df['sales_revenue'] = df['transaction_qty'] * df['unit_price']


# **EDA**

In [ ]:
#sales and weather by day
day_by_day_sales = df.groupby('transaction_date').agg({
    'sales_revenue': 'sum',
    'transaction_qty': 'sum',
    'temp': 'mean',
    'humidity': 'mean',
    'windspeed': 'mean',
    'visibility': 'mean',
    'precip': 'sum',
    'windgust': 'mean',
    'cloudcover': 'mean',
    'uvindex' : 'mean'
}).reset_index()


day_by_day_sales = day_by_day_sales.round(3)

In [ ]:
# Aggregate hourly sales revenue
df['transaction_datetime'] = pd.to_datetime(
    df['transaction_date'].astype(str) + ' ' + df['transaction_time']
)
df['hour'] = df['transaction_datetime'].dt.hour
hourly_sales = df.groupby(['transaction_date', 'hour']).agg({
    'sales_revenue': 'sum'
}).reset_index()

In [ ]:
# Aggregate daily weather from July-December dataset
daily_weather_july_dec = weather_data_july_dec.groupby('datetime').agg({
    'temp': 'mean',
    'humidity': 'mean',
    'precip': 'sum',
    'cloudcover': 'mean',
    'uvindex': 'mean',
    'solarradiation': 'mean'
}).reset_index()


In [ ]:
# Sales trend plot
plt.figure(figsize=(12, 6))
plt.plot(day_by_day_sales['transaction_date'], day_by_day_sales['sales_revenue'], marker='o', linestyle='-', alpha=0.7)
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.title('Daily Coffee Shop Sales Trend')
plt.xticks(rotation=45)
plt.show()




In [ ]:
# Correlation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(day_by_day_sales.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Between Sales and Weather Factors')
plt.show()

### **DATA PREPERATION FOR MODELLING**

In [ ]:

# Keep only the relevant weather features
july_dec_weather = weather_data_jul_dec[['transaction_date', 'temp', 'humidity', 'precip', 'windgust', 'cloudcover', 'uvindex']]

# Append July-December weather data to the existing dataset
full_weather_data = pd.concat([day_by_day_sales, july_dec_weather], ignore_index=True)


In [ ]:
# This is our target
y = day_by_day_sales[['sales_revenue']]

# These are our features/variables
X = day_by_day_sales[['temp', 'humidity', 'precip',  'cloudcover']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# # Split into train (Jan-June) and test (July-Dec)
# train_data = day_by_day_sales[day_by_day_sales['transaction_date'].dt.month <= 6]
# test_data = day_by_day_sales[day_by_day_sales['transaction_date'].dt.month > 6]


# X_train = train_data[['temp', 'humidity', 'precip', 'windgust', 'cloudcover']]
# y_train = train_data['sales_revenue']

# X_future = full_weather_data[full_weather_data['transaction_date'].dt.month > 6][['temp', 'humidity', 'precip', 'windgust', 'cloudcover']]

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae}, R2 Score: {r2}')

In [ ]:
print(daily_weather_july_dec.columns)


In [ ]:
# Predict sales revenue for July-December
X_july_dec = daily_weather_july_dec[['temp', 'humidity', 'precip', 'cloudcover']]
daily_weather_july_dec['predicted_sales_revenue'] = model.predict(X_july_dec)

In [ ]:
from sklearn.metrics import mean_absolute_error

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"MAE (Train): {mae_train}")
print(f"MAE (Test): {mae_test}")


from sklearn.metrics import r2_score

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"R² Score (Train): {r2_train}")
print(f"R² Score (Test): {r2_test}")


from sklearn.metrics import mean_squared_error
import numpy as np

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

print(f"RMSE (Train): {rmse_train}")
print(f"RMSE (Test): {rmse_test}")


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred_test, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')  # Perfect Fit Line
plt.xlabel("Actual Sales Revenue")
plt.ylabel("Predicted Sales Revenue")
plt.title("Actual vs Predicted Sales Revenue")
plt.show()


In [ ]:
residuals = y_test - y_pred_test

plt.scatter(y_pred_test, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Predicted Sales Revenue")
plt.ylabel("Residual (Error)")
plt.title("Residual Plot")
plt.show()


In [ ]:
print(X_future.shape)

In [ ]:
print(day_by_day_sales['transaction_date'].dt.month.unique())


### **TRAIN THE MODEL**

LINEAR REGRESSION


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_future_pred = model.predict(X_future)

predicted_sales = pd.DataFrame({
    'transaction_date': full_weather_data[full_weather_data['transaction_date'].dt.month > 6]['transaction_date'],
    'predicted_sales_revenue': y_future_pred
})

predicted_sales.to_csv("/content/drive/MyDrive/Data Mining Project/Data/Weather Data/Predicted_Sales_July_December.csv", index=False)
